In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("/sps/lsst/users/ebarroso/crow")
from crow.cluster_modules.shear_profile import *
from crow.recipes.binned_exact import ExactBinnedClusterRecipe
from crow.recipes.binned_grid import GridBinnedClusterRecipe
from crow.cluster_modules.mass_proxy import MurataUnbinned, MurataBinned
from crow.cluster_modules.kernel import SpectroscopicRedshift
from crow.cluster_modules.completeness_models import CompletenessAguena16
from crow.cluster_modules.purity_models import PurityAguena16

#from firecrown.models.cluster import ClusterProperty
from crow.properties import ClusterProperty
import time
import numpy as np

from scipy.integrate import dblquad, tplquad, simpson

In [3]:
import pyccl as ccl
hmf = ccl.halos.MassFuncTinker08(mass_def="200c")
cosmo = ccl.Cosmology(
    Omega_c=0.2607,      # Cold dark matter density
    Omega_b=0.04897,     # Baryon density
    h=0.6766,            # Hubble parameter
    sigma8=0.8102,       # Matter fluctuation amplitude
    n_s=0.9665,          # Spectral index
)
cl_delta_sigma = ClusterShearProfile(cosmo, hmf, 4.0, True)
#cl_delta_sigma.vectorized= True
pivot_mass, pivot_redshift = 14.625862906, 0.6
comp_dist = CompletenessAguena16()
pur_dist = PurityAguena16()
mass_distribution = MurataUnbinned(pivot_mass, pivot_redshift)
mass_distribution_binned = MurataBinned(pivot_mass, pivot_redshift)
redshift_distribution = SpectroscopicRedshift()



In [4]:
##### Parameters to be used in both recipes #####
mass_grid_size = 30
redshift_grid_size = 10
proxy_grid_size = 12
sky_area = 440
mass_interval = (12.5, 15.0)
cluster_theory = cl_delta_sigma
z_bin = (0.2, 0.4)
z_points = np.linspace(z_bin[0], z_bin[1], redshift_grid_size) 
proxy_bin = (1.0, 1.3)
proxy_points = np.linspace(proxy_bin[0], proxy_bin[1], proxy_grid_size)
radius_center = np.array([4.0])
#################################################

recipe_integral = ExactBinnedClusterRecipe(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_binned,
        completeness=comp_dist,
    )

recipe_grid = GridBinnedClusterRecipe(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
        completeness=comp_dist,
    proxy_grid_size=proxy_grid_size,
    redshift_grid_size=redshift_grid_size,
    mass_grid_size=mass_grid_size,
    )

In [5]:
print(recipe_integral._completeness_distribution(np.array([mass_interval[0]]),np.array([z_bin[0]])))

[0.00997866]


## Testing mass function

In [6]:
hmf_grid = recipe_grid._get_hmf_grid(z_points, sky_area, (z_points[0], z_points[-1]))
log_mass_grid = recipe_grid.log_mass_grid
def integrand(log_mass_scalar, z_scalar):
    """
    Inputs from dblquad are SCALARS:
    1. log_mass_scalar (y-variable in dblquad)
    2. z_scalar (x-variable in dblquad)
    """
    
    # 1. Convert scalars to single-element arrays to satisfy the methods
    z_array = np.array([z_scalar])
    log_mass_array = np.array([log_mass_scalar])

    # 2. Call the methods with the correct, named arrays
    # comoving_volume(z, sky_area)
    vol_array = recipe_integral.cluster_theory.comoving_volume(z_array, sky_area)
    
    # mass_function(log_mass, z)
    hmf_array = recipe_integral.cluster_theory.mass_function(log_mass_array, z_array)
    
    # 3. Return the result as a scalar float
    return (vol_array * hmf_array)[0]



integral_hmf, error_estimate = dblquad(
    func=integrand, 
    a=z_bin[0],       # lower limit for z (x-axis)
    b=z_bin[1],       # upper limit for z (x-axis)
    gfun=lambda x: mass_interval[0], # lower limit for log_mass (y-axis)
    hfun=lambda x: mass_interval[1]  # upper limit for log_mass (y-axis)
)
integral_over_mass = simpson(
    y=hmf_grid, 
    x=log_mass_grid, 
    axis=1
)

# Step B: Integrate over the redshift dimension (axis=0 of the new array)
# The result is the final scalar integrated number count
simpson_hmf = simpson(
    y=integral_over_mass, 
    x=z_points, 
    axis=0
)

print(f"Simpson Integral {simpson_hmf}, dblquad integral {integral_hmf}")
print(f"Abs error {abs(integral_hmf - simpson_hmf)}, rel error {abs(1.0 - simpson_hmf/integral_hmf)}")



Simpson Integral 87221.31147840535, dblquad integral 87220.6489541402
Abs error 0.6625242651498411, rel error 7.595956612371779e-06


## Testing mass-richness

In [7]:
mass_richness_grid = recipe_grid._get_mass_richness_grid(z_points, proxy_points, None)

def integrand(log_mass_scalar, z_scalar):
    """
    Inputs from dblquad are SCALARS:
    1. log_mass_scalar (y-variable in dblquad)
    2. z_scalar (x-variable in dblquad)
    """
    
    z_array = np.array([z_scalar])
    log_mass_array = np.array([log_mass_scalar])
    return mass_distribution_binned.distribution(log_mass_array, z_array, proxy_bin)    

z_grid_mesh, log_mass_grid_mesh = np.meshgrid(
            z_points, log_mass_grid, indexing='ij'
        )




integral_mass_richness, error_estimate = dblquad(
    func=integrand, 
    a=z_bin[0],       # lower limit for z (x-axis)
    b=z_bin[1],       # upper limit for z (x-axis)
    gfun=lambda x: mass_interval[0], # lower limit for log_mass (y-axis)
    hfun=lambda x: mass_interval[1]  # upper limit for log_mass (y-axis)
)

integral_over_mass = simpson(
    y=mass_richness_grid, 
    x=log_mass_grid, 
    axis=2
)

integral_over_proxy = simpson(
    y=integral_over_mass, 
    x=proxy_points * np.log(10.0), 
    axis=0
)

simpson_mass_richness = simpson(
    y=integral_over_proxy, 
    x=z_points, 
    axis=0
)

print(f"Simpson Integral {simpson_mass_richness}, dblquad integral {integral_mass_richness}")
print(f"Abs error {abs(integral_mass_richness - simpson_mass_richness)}, rel error {abs(1.0 - simpson_mass_richness/integral_mass_richness)}")



Simpson Integral 0.07492115089000648, dblquad integral 0.0749396525986431
Abs error 1.8501708636625702e-05, rel error 0.0002468881025605718


## Testing completeness

In [8]:
comp_grid = recipe_grid._get_completeness_grid(z_points, None)
def integrand(log_mass_scalar, z_scalar):
    z_array = np.array([z_scalar])
    log_mass_array = np.array([log_mass_scalar])
    return recipe_integral._completeness_distribution(log_mass_array, z_array)



integral_hmf, error_estimate = dblquad(
    func=integrand, 
    a=z_bin[0],       # lower limit for z (x-axis)
    b=z_bin[1],       # upper limit for z (x-axis)
    gfun=lambda x: mass_interval[0], # lower limit for log_mass (y-axis)
    hfun=lambda x: mass_interval[1]  # upper limit for log_mass (y-axis)
)
integral_over_mass = simpson(
    y=comp_grid, 
    x=log_mass_grid, 
    axis=1
)

# Step B: Integrate over the redshift dimension (axis=0 of the new array)
# The result is the final scalar integrated number count
simpson_hmf = simpson(
    y=integral_over_mass, 
    x=z_points, 
    axis=0
)

print(f"Simpson Integral {simpson_hmf}, dblquad integral {integral_hmf}")
print(f"Abs error {abs(integral_hmf - simpson_hmf)}, rel error {abs(1.0 - simpson_hmf/integral_hmf)}")



Simpson Integral 0.28510822270960434, dblquad integral 0.2851081640654921
Abs error 5.864411223299726e-08, rel error 2.0569075043219698e-07


## Testing Purity

In [9]:
######################## Recipe with purity ########################
#We do not use purity to test everywhere because the exact has no implementation yet#
recipe_grid_impure = GridBinnedClusterRecipe(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
        completeness=comp_dist,
    purity = pur_dist,
    proxy_grid_size=proxy_grid_size,
    redshift_grid_size=redshift_grid_size,
    mass_grid_size=mass_grid_size,
    )
########################
pur_grid = recipe_grid_impure._get_purity_grid(z_points, proxy_points, None)
print(f"\n\n",pur_grid.shape)

def integrand(ln_proxy_scalar, z_scalar):
    log_proxy_scalar = ln_proxy_scalar / np.log(10.0)
    z_array = np.array([z_scalar])
    log_proxy_array = np.array([log_proxy_scalar])
    return recipe_grid_impure._purity_distribution(z_array, log_proxy_array)


integral_hmf, error_estimate = dblquad(
    func=integrand, 
    a=z_bin[0],       # lower limit for z (x-axis)
    b=z_bin[1],       # upper limit for z (x-axis)
    gfun=lambda x: proxy_bin[0] * np.log(10.0), # lower limit for log_mass (y-axis)
    hfun=lambda x: proxy_bin[1] * np.log(10.0), # upper limit for log_mass (y-axis)
)
integral_over_proxy = simpson(
    y=pur_grid, 
    x=proxy_points * np.log(10.0), 
    axis=0
)

# Step B: Integrate over the redshift dimension (axis=0 of the new array)
# The result is the final scalar integrated number count
simpson_hmf = simpson(
    y=integral_over_proxy, 
    x=z_points, 
    axis=0
)

print(f"Simpson Integral {simpson_hmf}, dblquad integral {integral_hmf}")
print(f"Abs error {abs(integral_hmf - simpson_hmf)}, rel error {abs(1.0 - simpson_hmf/integral_hmf)}")





 (12, 10)
Simpson Integral 0.13497569756262148, dblquad integral 0.1349756810357927
Abs error 1.6526828794383164e-08, rel error 1.224430110635666e-07


## Testing Shear

In [10]:
shear_grid = recipe_grid._get_shear_grid(z_points, np.array(radius_center), None)
log_mass_grid = recipe_grid.log_mass_grid
def integrand(log_mass_scalar, z_scalar):
    """
    Inputs from dblquad are SCALARS:
    1. log_mass_scalar (y-variable in dblquad)
    2. z_scalar (x-variable in dblquad)
    """
    
    # 1. Convert scalars to single-element arrays to satisfy the methods
    z_array = np.array([z_scalar])
    log_mass_array = np.array([log_mass_scalar])
    shear = recipe_integral.cluster_theory.compute_shear_profile(log_mass_array, z_array, radius_center = radius_center)
    
    # 3. Return the result as a scalar float
    return shear[0]



integral_hmf, error_estimate = dblquad(
    func=integrand, 
    a=z_bin[0],       # lower limit for z (x-axis)
    b=z_bin[1],       # upper limit for z (x-axis)
    gfun=lambda x: mass_interval[0], # lower limit for log_mass (y-axis)
    hfun=lambda x: mass_interval[1]  # upper limit for log_mass (y-axis)
)
integral_over_mass = simpson(
    y=shear_grid, 
    x=log_mass_grid, 
    axis=1
)

# Step B: Integrate over the redshift dimension (axis=0 of the new array)
# The result is the final scalar integrated number count
simpson_hmf = simpson(
    y=integral_over_mass, 
    x=z_points, 
    axis=0
)

print(f"Simpson Integral {simpson_hmf}, dblquad integral {integral_hmf}")
print(f"Abs error {abs(integral_hmf - simpson_hmf)}, rel error {abs(1.0 - simpson_hmf/integral_hmf)}")

# print( recipe_integral.cluster_theory.compute_shear_profile(np.array([1.0,3.0])[:, None], np.array([1.0, 1.0, 2.0,1.0]), radius_center = np.array([2.0, 1.0,1.0])[:, None]))

# print(np.array([1.0,1.0])[:, None])
# print(np.array([2.0, 1.0,1.0])[:, None])

Simpson Integral [3.10754723e+12], dblquad integral 3107478462523.7495
Abs error [68772318.41601562], rel error [2.21312293e-05]


## Testing Counts

In [11]:
recipe_grid.setup()
t1 = time.time()
counts_grid = recipe_grid.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area)
t2 = time.time()
print(z_bin, proxy_bin)
counts_integral = recipe_integral.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area)
t21 = time.time()
t3 = time.time()
counts_grid_2 = recipe_grid.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area)
t4 = time.time()
recipe_grid.setup()
t5 = time.time()
counts_grid_3 = recipe_grid.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area)
t6 = time.time()
print(f"Simpson Integral {counts_grid}, dblquad integral {counts_integral}")
print(f"Abs error {abs(counts_integral - counts_grid)}, rel error {abs(1.0 - counts_grid/counts_integral)}")
print(f"First eval took: {t2-t1}, second eval took: {t4-t3}, integral took: {t21-t2}")
print(f"After reset: {t6-t5}")
print(f" Counts 1, 2 ,3: {counts_grid, counts_grid_2, counts_grid_3}")


(0.2, 0.4) (1.0, 1.3)
Simpson Integral 465.86054457432704, dblquad integral 465.88615450982513
Abs error 0.025609935498096092, rel error 5.4970372590390504e-05
First eval took: 0.0015347003936767578, second eval took: 0.0007061958312988281, integral took: 0.0427401065826416
After reset: 0.0011837482452392578
 Counts 1, 2 ,3: (np.float64(465.86054457432704), np.float64(465.86054457432704), np.float64(465.86054457432704))


## Testing DeltaSigma Profile

In [12]:
average_on = ClusterProperty.NONE
average_on |= ClusterProperty.DELTASIGMA
recipe_grid.setup()
t1 = time.time()
counts_grid = recipe_grid.evaluate_theory_prediction_lensing_profile(z_edges=z_bin, log_proxy_edges=proxy_bin, radius_centers=radius_center,sky_area=sky_area, average_on=average_on)
t2 = time.time()
print(radius_center)
counts_integral = recipe_integral.evaluate_theory_prediction_lensing_profile(z_edges=z_bin, log_proxy_edges=proxy_bin, radius_centers=radius_center, sky_area=sky_area,average_on=average_on)
print(counts_integral)
t21 = time.time()
t3 = time.time()
counts_grid_2 = recipe_grid.evaluate_theory_prediction_lensing_profile(z_edges=z_bin, log_proxy_edges=proxy_bin, radius_centers=radius_center, sky_area=sky_area,average_on=average_on)
t4 = time.time()
recipe_grid.setup()
t5 = time.time()
counts_grid_3 = recipe_grid.evaluate_theory_prediction_lensing_profile(z_edges=z_bin, log_proxy_edges=proxy_bin, radius_centers=radius_center, sky_area=sky_area,average_on=average_on)
t6 = time.time()
print(f"Simpson Integral {counts_grid}, dblquad integral {counts_integral}")
print(f"Abs error {abs(counts_integral - counts_grid)}, rel error {abs(1.0 - counts_grid/counts_integral)}")
print(f"First eval took: {t2-t1}, second eval took: {t4-t3}, integral took: {t21-t2}")
print(f"After reset: {t6-t5}")
print(f" Counts 1, 2 ,3: {counts_grid, counts_grid_2, counts_grid_3}")


[4.]
[3.67451479e+15]
Simpson Integral [3.67407585e+15], dblquad integral [3.67451479e+15]
Abs error [4.38941366e+11], rel error [0.00011946]
First eval took: 0.0026946067810058594, second eval took: 0.0008661746978759766, integral took: 0.11332583427429199
After reset: 0.0024514198303222656
 Counts 1, 2 ,3: (array([3.67407585e+15]), array([3.67407585e+15]), array([3.67407585e+15]))


## Testing Reduced Shear Profile

In [13]:
cl_reduced_shear = ClusterShearProfile(cosmo, hmf, 4.0, False, True)
cl_reduced_shear.set_beta_parameters(10)
cl_reduced_shear.set_beta_s_interp(1.1, 1.3)
recipe_integral_reduced = ExactBinnedClusterRecipe(
        mass_interval=mass_interval,
        cluster_theory=cl_reduced_shear,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_binned,
        #completeness=comp_dist,
    )

recipe_grid_reduced = GridBinnedClusterRecipe(
        mass_interval=mass_interval,
        cluster_theory=cl_reduced_shear,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
        #completeness=comp_dist,
    proxy_grid_size=proxy_grid_size,
    redshift_grid_size=redshift_grid_size,
    mass_grid_size=mass_grid_size,
    )

In [14]:
average_on = ClusterProperty.NONE
average_on |= ClusterProperty.DELTASIGMA
recipe_grid.setup()
t1 = time.time()
counts_grid = recipe_grid_reduced.evaluate_theory_prediction_lensing_profile(z_edges=z_bin, log_proxy_edges=proxy_bin, radius_centers=radius_center,sky_area=sky_area, average_on=average_on)
t2 = time.time()
counts_integral = recipe_integral_reduced.evaluate_theory_prediction_lensing_profile(z_edges=z_bin, log_proxy_edges=proxy_bin, radius_centers=radius_center, sky_area=sky_area,average_on=average_on)
t21 = time.time()
t3 = time.time()
counts_grid_2 = recipe_grid_reduced.evaluate_theory_prediction_lensing_profile(z_edges=z_bin, log_proxy_edges=proxy_bin, radius_centers=radius_center, sky_area=sky_area,average_on=average_on)
t4 = time.time()
recipe_grid.setup()
t5 = time.time()
counts_grid_3 = recipe_grid_reduced.evaluate_theory_prediction_lensing_profile(z_edges=z_bin, log_proxy_edges=proxy_bin, radius_centers=radius_center, sky_area=sky_area,average_on=average_on)
t6 = time.time()
print(f"Simpson Integral {counts_grid}, dblquad integral {counts_integral}")
print(f"Abs error {abs(counts_integral - counts_grid)}, rel error {abs(1.0 - counts_grid/counts_integral)}")
print(f"First eval took: {t2-t1}, second eval took: {t4-t3}, integral took: {t21-t2}")
print(f"After reset: {t6-t5}")
print(f" Values 1, 2 ,3: {counts_grid, counts_grid_2, counts_grid_3}")


Simpson Integral [1.22802065], dblquad integral [1.36087356]
Abs error [0.13285291], rel error [0.09762325]
First eval took: 0.04190349578857422, second eval took: 0.0007469654083251953, integral took: 0.28732752799987793
After reset: 0.0007152557373046875
 Values 1, 2 ,3: (array([1.22802065]), array([1.22802065]), array([1.22802065]))


## Testing Mean Log Mass Profile

In [15]:
average_on_mass = ClusterProperty.NONE
average_on_mass |= ClusterProperty.MASS

recipe_grid.setup()
t1 = time.time()
counts_grid = recipe_grid.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area, average_on_mass)
t2 = time.time()
print(z_bin, proxy_bin)
counts_integral = recipe_integral.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area, average_on_mass)
t21 = time.time()
t3 = time.time()
counts_grid_2 = recipe_grid.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area, average_on_mass)
t4 = time.time()
recipe_grid.setup()
t5 = time.time()
counts_grid_3 = recipe_grid.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area, average_on_mass)
t6 = time.time()
print(f"Simpson Integral {counts_grid}, dblquad integral {counts_integral}")
print(f"Abs error {abs(counts_integral - counts_grid)}, rel error {abs(1.0 - counts_grid/counts_integral)}")
print(f"First eval took: {t2-t1}, second eval took: {t4-t3}, integral took: {t21-t2}")
print(f"After reset: {t6-t5}")
print(f" Counts 1, 2 ,3: {counts_grid, counts_grid_2, counts_grid_3}")


(0.2, 0.4) (1.0, 1.3)
Simpson Integral 6643.852449550316, dblquad integral 6644.228454470479
Abs error 0.3760049201628135, rel error 5.659120885737057e-05
First eval took: 0.0013675689697265625, second eval took: 0.0006885528564453125, integral took: 0.0020668506622314453
After reset: 0.0012717247009277344
 Counts 1, 2 ,3: (np.float64(6643.852449550316), np.float64(6643.852449550316), np.float64(6643.852449550316))


## Testing Mean redshift Profile

In [16]:
average_on_mass = ClusterProperty.NONE
average_on_mass |= ClusterProperty.REDSHIFT

recipe_grid.setup()
t1 = time.time()
counts_grid = recipe_grid.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area, average_on_mass)
t2 = time.time()
print(z_bin, proxy_bin)
counts_integral = recipe_integral.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area, average_on_mass)
t21 = time.time()
t3 = time.time()
counts_grid_2 = recipe_grid.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area, average_on_mass)
t4 = time.time()
recipe_grid.setup()
t5 = time.time()
counts_grid_3 = recipe_grid.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area, average_on_mass)
t6 = time.time()
print(f"Simpson Integral {counts_grid}, dblquad integral {counts_integral}")
print(f"Abs error {abs(counts_integral - counts_grid)}, rel error {abs(1.0 - counts_grid/counts_integral)}")
print(f"First eval took: {t2-t1}, second eval took: {t4-t3}, integral took: {t21-t2}")
print(f"After reset: {t6-t5}")
print(f" Counts 1, 2 ,3: {counts_grid, counts_grid_2, counts_grid_3}")


(0.2, 0.4) (1.0, 1.3)
Simpson Integral 145.5788864174151, dblquad integral 145.5957901462481
Abs error 0.016903728833000287, rel error 0.00011610039559539764
First eval took: 0.0013070106506347656, second eval took: 0.0006926059722900391, integral took: 0.0016515254974365234
After reset: 0.0012030601501464844
 Counts 1, 2 ,3: (np.float64(145.5788864174151), np.float64(145.5788864174151), np.float64(145.5788864174151))


## Test speed


In [17]:
##### Parameters to be used in both recipes #####
mass_grid_size = 40
redshift_grid_size = 10
proxy_grid_size = 10
sky_area = 440
mass_interval = (12.0, 17.0)
cluster_theory = cl_delta_sigma
z_bin = (0.2, 0.4)
z_points = np.linspace(z_bin[0], z_bin[1], redshift_grid_size) 
proxy_bin = (1.0, 1.3)
proxy_points = np.linspace(proxy_bin[0], proxy_bin[1], proxy_grid_size)
radius_center = np.array([4.0])
average_on_counts = ClusterProperty.NONE
average_on_shear = ClusterProperty.NONE
average_on_shear |= ClusterProperty.DELTASIGMA
#################################################

recipe_integral_speed = ExactBinnedClusterRecipe(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution_binned,
        #completeness=comp_dist,
    )

recipe_grid_speed = GridBinnedClusterRecipe(
        mass_interval=mass_interval,
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
        #completeness=comp_dist,
    proxy_grid_size=proxy_grid_size,
    redshift_grid_size=redshift_grid_size,
    mass_grid_size=mass_grid_size,
    )
recipe_grid_speed.setup()
counts_grid = recipe_grid_speed.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area,average_on_counts)
counts_integral = recipe_integral_speed.evaluate_theory_prediction_counts(z_bin, proxy_bin, sky_area, average_on_counts)

counts_grid = recipe_grid_speed.evaluate_theory_prediction_lensing_profile(z_bin, proxy_bin, radius_center,sky_area, average_on_shear)
t2 = time.time()
counts_integral = recipe_integral_speed.evaluate_theory_prediction_lensing_profile(z_bin, proxy_bin, radius_center, sky_area,average_on_shear)
t21 = time.time()


In [18]:
import time
import numpy as np

# Your parameters
mass_grid_size = 40
redshift_grid_size = 12
proxy_grid_size = 12
sky_area = 440
mass_interval = (12.0, 17.0)
z_bin = (0.2, 0.4)
proxy_bin = (1.0, 1.3)
radius_center = np.array([4.0])

average_on_counts = ClusterProperty.NONE
average_on_shear  = ClusterProperty.DELTASIGMA

# Recipes
recipe_integral_speed = ExactBinnedClusterRecipe(
    mass_interval=mass_interval,
    cluster_theory=cl_delta_sigma,
    redshift_distribution=redshift_distribution,
    mass_distribution=mass_distribution_binned,
    completeness=comp_dist,
)

recipe_grid_speed = GridBinnedClusterRecipe(
    mass_interval=mass_interval,
    cluster_theory=cl_delta_sigma,
    redshift_distribution=redshift_distribution,
    mass_distribution=mass_distribution,
    completeness=comp_dist,
    proxy_grid_size=proxy_grid_size,
    redshift_grid_size=redshift_grid_size,
    mass_grid_size=mass_grid_size,
)

# ---- One-shot timing: counts ----
recipe_grid_speed.setup()

t0 = time.time()
counts_grid = recipe_grid_speed.evaluate_theory_prediction_counts(
    z_bin, proxy_bin, sky_area, average_on_counts
)
t1 = time.time()

counts_integral = recipe_integral_speed.evaluate_theory_prediction_counts(
    z_bin, proxy_bin, sky_area, average_on_counts
)
t2 = time.time()

print("\n--- One-shot counts ---")
print("Grid time:    %.4f s" % (t1 - t0))
print("Integral time: %.4f s" % (t2 - t1))


# ---- One-shot timing: shear ----
t0 = time.time()
shear_grid = recipe_grid_speed.evaluate_theory_prediction_lensing_profile(
    z_bin, proxy_bin, radius_center, sky_area, average_on_shear
)
t1 = time.time()

shear_integral = recipe_integral_speed.evaluate_theory_prediction_lensing_profile(
    z_bin, proxy_bin, radius_center, sky_area, average_on_shear
)
t2 = time.time()

print("\n--- One-shot shear ---")
print("Grid time:    %.4f s" % (t1 - t0))
print("Integral time: %.4f s" % (t2 - t1))



--- One-shot counts ---
Grid time:    0.0523 s
Integral time: 0.0023 s

--- One-shot shear ---
Grid time:    0.0021 s
Integral time: 0.4202 s


In [19]:
richness_bins = [(0.3,0.4), (0.4,0.6), (0.6,0.8), (0.8,1.0)]   # example
proxy_bins    = [(1.0,1.1), (1.1,1.2), (1.2,1.3), (1.3,1.4), (1.4,1.5)]
radii         = np.linspace(1.0, 10.0, 4)

# ----------------------------------------------
# Total GRID computation time
# ----------------------------------------------
recipe_grid_speed.setup()
counts_grid_arr = []
shear_grid_arr = []
tg0 = time.time()
for rbin in richness_bins:
    for pbin in proxy_bins:
        counts_grid_arr.append(recipe_grid_speed.evaluate_theory_prediction_counts(rbin, pbin, sky_area, average_on_counts))
        shear_grid_arr.append(recipe_grid_speed.evaluate_theory_prediction_lensing_profile(rbin, pbin, radii, sky_area, average_on_shear))
tg1 = time.time()
print("\nTOTAL GRID TIME = %.2f s" % (tg1 - tg0))


# ----------------------------------------------
# Total INTEGRAL computation time
# ----------------------------------------------
counts_int_arr = []
shear_int_arr = []
ti0 = time.time()
for rbin in richness_bins:
    for pbin in proxy_bins:
        counts_int_arr.append(recipe_integral_speed.evaluate_theory_prediction_counts(rbin, pbin, sky_area, average_on_counts))
        shear_int_arr.append(recipe_integral_speed.evaluate_theory_prediction_lensing_profile(rbin, pbin, radii, sky_area, average_on_shear))
ti1 = time.time()
print("TOTAL INTEGRAL TIME = %.2f s" % (ti1 - ti0))

# ----------------------------------------------
# Comparison of Results
# ----------------------------------------------

# Convert lists to NumPy arrays for easy calculation
counts_grid = np.array(counts_grid_arr).flatten()
counts_int = np.array(counts_int_arr).flatten()
shear_grid = np.array(shear_grid_arr).flatten()
shear_int = np.array(shear_int_arr).flatten()

counts_diff = np.where(
    counts_int != 0.0, 
    np.abs(counts_grid - counts_int) / np.abs(counts_int), 
    0.0
)
rms_counts_diff = np.sqrt(np.mean(counts_diff**2))

shear_diff = np.where(
    shear_int != 0.0, 
    np.abs(shear_grid - shear_int) / np.abs(shear_int), 
    0.0
)
rms_shear_diff = np.sqrt(np.mean(shear_diff**2))

# Final Comparison Printout
print("\n--- RESULTS COMPARISON ---")
print("Total number of bins computed: %d" % len(counts_grid))
print("RMS Relative Diff (Counts): %.4e" % rms_counts_diff)
print("RMS Relative Diff (Shear):  %.4e" % rms_shear_diff)
print("--------------------------")



TOTAL GRID TIME = 0.23 s
TOTAL INTEGRAL TIME = 24.13 s

--- RESULTS COMPARISON ---
Total number of bins computed: 20
RMS Relative Diff (Counts): 3.2608e-04
RMS Relative Diff (Shear):  3.7853e-04
--------------------------


In [ ]:
def frac_diff(a, b):
    return np.abs(a - b) / (np.abs(b) + 1e-12)


print("\n===== BEGIN ACCURACY SWEEP WITH COUNTS =====\n")

# --- ranges to test ---
proxy_sweep    = [2, 5, 10, 20, 30, 40, 60, 80, 120, 160]
z_sweep        = [2, 5, 10, 20, 30, 40]
mass_sweep     = [5, 10, 20, 40, 60, 80, 120]

# --- fixed test bin ---
z_lo, z_hi = z_bin
proxy_lo, proxy_hi = proxy_bin
R = np.array(radius_center)

print("Test bin:")
print("  z = (%.3f, %.3f)" % z_bin)
print("  proxy = (%.3f, %.3f)" % proxy_bin)
print("  R = %.3f" % R.item())


# --------------------------------------------------------------
# 1) PROXY SWEEP
# --------------------------------------------------------------
print("\n---- Sweep proxy_grid_size ----")

for pts in proxy_sweep:
    print(f"\nTrying proxy_grid_size = {pts}")

    recipe_grid_speed_pts = GridBinnedClusterRecipe(
        mass_interval=mass_interval,
        cluster_theory=cl_delta_sigma,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
        completeness=comp_dist,
        proxy_grid_size=pts,
        redshift_grid_size=redshift_grid_size,
        mass_grid_size=mass_grid_size,
    )
    recipe_grid_speed_pts.setup()
    
    # Shear evaluation
    val_shear = recipe_grid_speed_pts.evaluate_theory_prediction_lensing_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    
    # Counts evaluation
    counts_val = recipe_grid_speed_pts.evaluate_theory_prediction_counts(
        z_bin, proxy_bin, sky_area
    )
    
    recipe_grid_speed_pts.setup()
    
    # Reference shear
    ref_shear = recipe_integral_speed.evaluate_theory_prediction_lensing_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    
    # Reference counts
    ref_counts = recipe_integral_speed.evaluate_theory_prediction_counts(
        z_bin, proxy_bin, sky_area
    )
    
    diff_shear  = frac_diff(val_shear, ref_shear).item()
    diff_counts = frac_diff(counts_val, ref_counts).item()

    print("  shear diff  = %.6f" % diff_shear)
    print("  counts diff = %.6f" % diff_counts)

    if diff_shear < 0.01 and diff_counts < 0.01:
        print("  → Achieved ≤1% accuracy at proxy_grid_size =", pts)
        break


# --------------------------------------------------------------
# 2) REDSHIFT SWEEP
# --------------------------------------------------------------
print("\n---- Sweep redshift_grid_size ----")

for pts in z_sweep:
    print(f"\nTrying redshift_grid_size = {pts}")

    recipe_grid_speed_pts = GridBinnedClusterRecipe(
        mass_interval=mass_interval,
        cluster_theory=cl_delta_sigma,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
        completeness=comp_dist,
        proxy_grid_size=proxy_grid_size,
        redshift_grid_size=pts,
        mass_grid_size=mass_grid_size,
    )
    recipe_grid_speed_pts.setup()
    
    val_shear = recipe_grid_speed_pts.evaluate_theory_prediction_lensing_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    counts_val = recipe_grid_speed_pts.evaluate_theory_prediction_counts(
        z_bin, proxy_bin, sky_area
    )

    ref_shear = recipe_integral_speed.evaluate_theory_prediction_lensing_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    ref_counts = recipe_integral_speed.evaluate_theory_prediction_counts(
        z_bin, proxy_bin, sky_area
    )
    
    diff_shear  = frac_diff(val_shear, ref_shear).item()
    diff_counts = frac_diff(counts_val, ref_counts).item()
    
    print("  shear diff  = %.6f" % diff_shear)
    print("  counts diff = %.6f" % diff_counts)

    if diff_shear < 0.01 and diff_counts < 0.01:
        print("  → Achieved ≤1% accuracy at redshift_grid_size =", pts)
        break


# --------------------------------------------------------------
# 3) MASS SWEEP
# --------------------------------------------------------------
print("\n---- Sweep mass_grid_size ----")

for pts in mass_sweep:
    print(f"\nTrying mass_grid_size = {pts}")

    recipe_grid_speed_pts = GridBinnedClusterRecipe(
        mass_interval=mass_interval,
        cluster_theory=cl_delta_sigma,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
        completeness=comp_dist,
        proxy_grid_size=proxy_grid_size,
        redshift_grid_size=redshift_grid_size,
        mass_grid_size=pts,
    )
    recipe_grid_speed_pts.setup()
    
    val_shear = recipe_grid_speed_pts.evaluate_theory_prediction_lensing_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    counts_val = recipe_grid_speed_pts.evaluate_theory_prediction_counts(
        z_bin, proxy_bin,  sky_area
    )

    ref_shear = recipe_integral_speed.evaluate_theory_prediction_lensing_profile(
        z_bin, proxy_bin, R, sky_area, average_on_shear
    )
    ref_counts = recipe_integral_speed.evaluate_theory_prediction_counts(
        z_bin, proxy_bin, sky_area
    )
    
    diff_shear  = frac_diff(val_shear, ref_shear).item()
    diff_counts = frac_diff(counts_val, ref_counts).item()
    
    print("  shear diff  = %.6f" % diff_shear)
    print("  counts diff = %.6f" % diff_counts)

    # Optional early break if both ≤1%
    # if diff_shear < 0.01 and diff_counts < 0.01:
    #     print("  → Achieved ≤1% accuracy at mass_grid_size =", pts)
    #     break


print("\n===== END ACCURACY SWEEP WITH COUNTS =====\n")



===== BEGIN ACCURACY SWEEP WITH COUNTS =====

Test bin:
  z = (0.200, 0.400)
  proxy = (1.000, 1.300)
  R = 4.000

---- Sweep proxy_grid_size ----

Trying proxy_grid_size = 2
  shear diff  = 0.009079
  counts diff = 0.122054

Trying proxy_grid_size = 5
  shear diff  = 0.000024
  counts diff = 0.000089
  → Achieved ≤1% accuracy at proxy_grid_size = 5

---- Sweep redshift_grid_size ----

Trying redshift_grid_size = 2
  shear diff  = 0.029323
  counts diff = 0.035479

Trying redshift_grid_size = 5
  shear diff  = 0.000047
  counts diff = 0.000107
  → Achieved ≤1% accuracy at redshift_grid_size = 5

---- Sweep mass_grid_size ----

Trying mass_grid_size = 5
  shear diff  = 0.141789
  counts diff = 0.229839

Trying mass_grid_size = 10
  shear diff  = 0.226844
  counts diff = 0.168378

Trying mass_grid_size = 20
  shear diff  = 0.045500
  counts diff = 0.008294

Trying mass_grid_size = 40
  shear diff  = 0.000041
  counts diff = 0.000096

Trying mass_grid_size = 60
  shear diff  = 0.000007
 